In [1]:
from tensorflow import keras
import numpy as np
from matplotlib import pyplot as plt
import cv2
from pathlib import Path

In [2]:
paths = ["train/image", "test/image"] 
images = []  # Lista para almacenar las imágenes

for path in paths:
    imgs = Path(path).glob("**/*.png") 
    imgs = [str(x) for x in imgs]
    for img in imgs: 
        imagen = cv2.imread(img,1)
        imagen = cv2.resize(imagen, (160, 160), cv2.INTER_AREA)
        imagen = imagen / 255
        images.append(imagen)

images = np.array(images)
print(f"Mask images shape: {images.shape}")
    

Mask images shape: (100, 160, 160, 3)


In [3]:
paths = ["train/mask", "test/mask"] 
masks = []  # Lista para almacenar las imágenes

for path in paths:
    imgs = Path(path).glob("**/*.png") 
    imgs = [str(x) for x in imgs]
    for img in imgs: 
        imagen = cv2.imread(img,0)
        imagen = cv2.resize(imagen, (160, 160), cv2.INTER_AREA)
        imagen = imagen / 255
        masks.append(imagen)

masks = np.array(masks)
print(f"Images shape: {masks.shape}")
    

Images shape: (100, 160, 160)


In [4]:
def get_model(img_size=(160,160,3), num_clases = 1) :
    inputs = keras.Input(shape=img_size)
    #Primera mitas de la red: Extracción de las características

    #Bloque de entrada
    x =  keras.layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation("relu")(x)

    #Parte residual
    previous_block = x

    #Bloques de convoluciEl bloque 1, 2 y 3 son iguales
    for filters in [64, 128, 256] : 
        x = keras.layers.Activation("relu")(x)
        x = keras.layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Activation("relu")(x)
        x = keras.layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Projección residual
        residual = keras.layers.Conv2D(filters, 1, strides=2, padding="same")(previous_block)
        x = keras.layers.add([x, residual])
        previous_block = x

    #Segunda mitad
    for filters in [256, 128, 64, 32] : 
        x = keras.layers.Activation("relu")(x)
        x = keras.layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Activation("relu")(x)
        x = keras.layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.UpSampling2D(2)(x)
        
        # Projección residual
        residual = keras.layers.UpSampling2D(2)(previous_block)
        residual = keras.layers.Conv2D(filters, 1, padding = "Same")(residual)
        x = keras.layers.add([x, residual])
        previous_block = x
    
    outputs = keras.layers.Conv2D(num_clases, 3, activation = "sigmoid", padding = "same")(x)
    
    # Definir el modelo
    model = keras.Model(inputs, outputs)
    return model

model = get_model()
print(model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 160, 160,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 80, 80,    │        896 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 80, 80,    │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 80, 80,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 80, 80,    │          0 │ activation[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d    │ (None, 80, 80,    │      2,400 │ activation_1[0][… │
│ (SeparableConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 80, 80,    │        256 │ separable_conv2d… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 80, 80,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_1  │ (None, 80, 80,    │      4,736 │ activation_2[0][… │
│ (SeparableConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 80, 80,    │        256 │ separable_conv2d… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 40, 40,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 40, 40,    │      2,112 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 40, 40,    │          0 │ max_pooling2d[0]… │
│                     │ 64)               │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 40, 40,    │          0 │ add[0][0]         │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_2  │ (None, 40, 40,    │      8,896 │ activation_3[0][… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 40, 40,    │        512 │ separable_conv2d… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 40, 40,    │          0 │ batch_normalizat

 Total params: 2,058,401 (7.85 MB)

 Trainable params: 2,054,625 (7.84 MB)

 Non-trainable params: 3,776 (14.75 KB)

None


In [5]:
model.compile(loss=keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam())
history = model.fit(images, masks, epochs=100, batch_size=32)

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - loss: 2.1750
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 866ms/step - loss: 1.3009
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 878ms/step - loss: 1.0703
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 868ms/step - loss: 0.8057
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 853ms/step - loss: 0.5086
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 862ms/step - loss: 0.3697
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 862ms/step - loss: 0.3199
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 867ms/step - loss: 0.2931
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 857ms/step - loss: 0.2814
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 810ms/step - loss: 0.2712
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 847ms/step - loss: 0.2685
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 838ms/step - loss: 0.2628
Epoch 13/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 814ms/step - loss: 0.2621
Epoch 14/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 801ms/step - loss: 0.2611
Epoch 15/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 776ms/step - loss: 0.2594
Epoch 